In [ ]:
import requests
import pandas as pd

def get_climate_data(api_token, state, start_date, end_date, variables):
    """
    Fetch climate data from NOAA API and calculate mean, max, min for specified variables.

    Args:
        api_token (str): NOAA API token.
        state (str): State name or bounding box in the format [min_lon, min_lat, max_lon, max_lat].
        start_date (str): Start date in YYYY-MM-DD format.
        end_date (str): End date in YYYY-MM-DD format.
        variables (list): List of variables to include (e.g., 'TMAX', 'TMIN', 'PRCP').

    Returns:
        dict: Dictionary containing mean, max, min for each variable.
    """
    base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"
    headers = {"token": api_token}
    params = {
        "datasetid": "GHCND",  # Global Historical Climatology Network Daily
        "locationid": f"FIPS:{state}",  # FIPS code for the state
        "startdate": start_date,
        "enddate": end_date,
        # "datatypeid": ",".join(variables),
        "units": "metric",
        "limit": 1000,  # Max number of results per request
    }

    response = requests.get(base_url, headers=headers, params=params)
    
    if response.status_code != 200:
        raise Exception(f"Error fetching data: {response.status_code} {response.text}")
    
    data = response.json().get("results", [])
    
    if not data:
        return "No data available for the specified parameters."
    
    # Convert data to a DataFrame
    df = pd.DataFrame(data)
    
    # Ensure 'value' column is numeric
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    
    # Calculate statistics
    results = {}
    for variable in variables:
        var_data = df[df['datatype'] == variable]
        results[variable] = {
            "mean": var_data['value'].mean(),
            "max": var_data['value'].max(),
            "min": var_data['value'].min()
        }
    
    return results

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
api_token = os.getenv("api_token")

state = "25"  # FIPS code for Massachusetts
start_date = "2023-01-01"
end_date = "2023-12-31"
variables = ["TMAX", "TMIN", "PRCP"]  # Example variables: max temp, min temp, precipitation

climate_stats = get_climate_data(api_token, state, start_date, end_date, variables)